Before you turn this problem in, make sure everything runs as expected. First, **restart the kernel** (in the menubar, select Kernel$\rightarrow$Restart) and then **run all cells** (in the menubar, select Cell$\rightarrow$Run All).

Make sure you fill in any place that says `YOUR CODE HERE` or "YOUR ANSWER HERE", as well as your name and collaborators below:

In [ ]:
NAME = ""
COLLABORATORS = ""

---

In [ ]:
from __future__ import print_function
%matplotlib inline
%precision 16
import numpy
import matplotlib.pyplot as plt
from scipy.integrate import solve_ivp

# HW 6b:  ODE Methods - Initial Value Problems - Stability and Stiffness



## Question 1 - Absolute Stability Regions and Order Stars

**(a)** [10] Single Step Schemes:
    
Derive $R(z)$ for the following single step schemes
* 2nd order Taylor Series method
* 4th order Taylor Series method
* RK2
* RK4

and comment on your solutions.

YOUR ANSWER HERE

**(b)** [8] Plot the regions of absolute stability for  Taylor Series methods of order 2-5

You may use the plotting code from the course notes

In [ ]:
# YOUR CODE HERE
raise NotImplementedError()

**(c)** [12] Linear Multi-Step schemes

Calculate the stability polynomial and plot the stability regions for
1. 3-step and 4-step Adams-Basforth methods, and
1. 3-step and 4-step Adams-Moulton methods.


And comment on the relative size of the stability regions for $RK4$, $AB4$ and $AM4$.
You may use the plotting code from the course notes

YOUR ANSWER HERE

In [ ]:
# YOUR CODE HERE
raise NotImplementedError()

**(d)** [8] The region of absolute stability as defined ensures that the error does not grow in subsequent time-steps hence why we require $|R(z)| < 1$.  In reality what we really want is that the errors decay faster than the solution or that it grows slower than the true solution.  Consider the solution of an ODE $u(t) = e^{\lambda t}$, if $\lambda < 0$ then we want our errors made at each time step $E^n$ to decay faster than $e^{\lambda t}$ so that $|E^n| < e^{\lambda t}$.  Conversely if $\lambda > 0$ then we might also want $|E^n| < e^{\lambda t}$.  This suggests a new definition of stability, called the **relative stability** defined as

$$|R(z)| \leq |e^z|.$$

These turn out to be difficult to plot but proved pivotal due to work in 1978 by Wanner, Hairer and Nørset.  More recently a new set of regions, called **order stars**, have become more popular to consider.  These are defined as the three regions

$$\begin{aligned}
    \mathcal{A}_{-} &= \{ z \in \mathbb{C}: |R(z)| < |e^z| \} = \{z \in \mathbb{C}: |e^{-z} R(z)| < 1\}, \\ 
    \mathcal{A}_{0} &= \{ z \in \mathbb{C}: |R(z)| = |e^z| \} = \{z \in \mathbb{C}: |e^{-z} R(z)| = 1\}, ~~~~\text{and} \\
    \mathcal{A}_{+} &= \{ z \in \mathbb{C}: |R(z)| > |e^z| \} = \{z \in \mathbb{C}: |e^{-z} R(z)| > 1\}.
\end{aligned}$$

Modify your plotting routines above to plot order stars and identify all of the different regimes for the Taylor series methods asked for in part (a).  Do you see any relation to the number of "fingers" and the order of the method?

In [ ]:
# YOUR CODE HERE
raise NotImplementedError()

YOUR ANSWER HERE

### Question 2 - stiff equations

Consider a simplified radioactive decay chain  that transforms a radioactive nuclide (like $^{235}$U) to a stable nuclide (like $^{207}$Pb) (for the real thing see [this](http://metadata.berkeley.edu/nuclear-forensics/Decay%20Chains.html))

$$
    u_1 \overset{K_1}{\rightarrow} u_2 \overset{K_2}{\rightarrow} u_3 \overset{K_3}{\rightarrow} u_4
$$
represented by the system of ODEs
\begin{align*}
    \frac{\text{d}u_1}{\text{d}t} &= -K_1 u_1 \\
    \frac{\text{d}u_2}{\text{d}t} &= K_1 u_1 - K_2 u_2 \\
    \frac{\text{d}u_3}{\text{d}t} &= K_2 u_2 - K_3 u_3 \\
    \frac{\text{d}u_4}{\text{d}t} &= K_3 u_3 \\
\end{align*}

where $\lambda_i$ are the decay constants for each nuclide (with the standard understanding that the decay constants are reported as $> 0$)

**(a)** rewrite this problem as a linear dynamical system

$$
    \frac{d\mathbf{u}}{dt} = A\mathbf{u}
$$

where 

$$
    \mathbf{u} = \begin{bmatrix} u_1 \\ u_2 \\ u_3 \\u_4 \\ \end{bmatrix}
$$

and $A$ is a matrix of constant coefficients.  Answer the following questions

1. what is the structure of the matrix $A$?
- what are the eigenvalues of the matrix $A$?
- what is the general analytic solution of the initial value problem 
$$
    \frac{d\mathbf{u}}{dt} = A\mathbf{u}\quad \mathbf{u}(0) = \mathbf{u}_0
$$
for arbitrary initial condition $\mathbf{u}_0$. **NOTE:** (just assume the eigenvectors exist, you don't need to compute them although they can be written analytically, **extra credit** for the full solution)

    

YOUR ANSWER HERE

**(a)** [5] Write a general function  `setA(K)`,  to assemble a $N\times N$ matrix $A$ given an  array of $N-1$ decay constants.


In [ ]:
def setA(K):
    """ set the Decay Matrix A, given decay constants K_i i=1,N
    
    Parameters
    ----------
        K: numpy array of decay constants [ K_1, K_2, ... K_N-1 ]
            assumes the final decay constant is 0
        
    Returns
    -------
        A: numpy 2d array of size N x N where N = len(K) + 1
    """
    # YOUR CODE HERE
    raise NotImplementedError()
    return A

In [ ]:
K = numpy.array([ 1.0, 2.0, 3.0 ])
# test setA
A = setA(K)
print('A = \n{}'.format(A))

u_0 = numpy.array([1.0, 0.0, 0.0, 0.0])
answer = numpy.array([-1.0, 1.0, 0., 0.])
numpy.testing.assert_allclose(A.dot(u_0), answer)
print('Passed setA test')

**(b)** Write a function `solve_decay_system` that uses `setA` and `scipy.integrate.solve_ivp` to compute the solution to the system of ODEs given an initial condition and time points to output at.  This function should take in the time points for the output, an initial condition, the ODE integrator to use (default to "RK45"), and the reaction rates $K_1$, $K_2$, and $K_3$.
You'll want to set `dense_output=True`, for use in part (b)

In [ ]:
def solve_decay_system(t_span, u_0,  K , method="RK45"):
    """ wrapper function to solve a 4 nuclide decay chain using scipy.integrate.solve_ivp
    
    parameters:
    -----------
    t_span: arraylike
        time range of integration t_span = [ t_min, t_max]
    u_0: ndarray of length 4
        numpy array of initial conditions 
    K: ndarray of length 3
    method: String
        a string containing one of the integration schemes available in solve_ivp
        one of "RK45", "RK23", "Radau", "BDF", "LSODA"
    """
    # YOUR CODE HERE
    raise NotImplementedError()
    return sol

In [ ]:
t_span = numpy.array([0.,1.])
sol = solve_decay_system(t_span, u_0, K, method='RK45')
t_test = numpy.linspace(0.,1.,10)
U_sol = numpy.array([[1.                , 0.8948393163712829, 0.800737401761885 ,
        0.7165313047527748, 0.6411802756034415, 0.5737534285773721,
        0.5134168033756287, 0.4594256588569134, 0.4111123936333331,
        0.3678795370466755],
       [0.                , 0.094101921174881 , 0.1595570258526453,
        0.2031143245142019, 0.2300709213438262, 0.2445598261804927,
        0.2498262704968908, 0.2483567965861113, 0.2420949351732015,
        0.2325408759062688],
       [0.                , 0.0098958300342681, 0.0317938394434248,
        0.0575760898841458, 0.0825409265439265, 0.1042479895430072,
        0.1215393110543409, 0.1342463444695756, 0.142589521806582 ,
        0.1470100078082271],
       [0.                , 0.001162932419568 , 0.0079117329420449,
        0.0227782808488774, 0.0462078765088057, 0.077438755699128 ,
        0.1152176150731396, 0.1579712000873996, 0.2042031493868833,
        0.2525695792388284]])
numpy.testing.assert_allclose(sol.sol(t_test), U_sol)
print("Success!")

**(b)** [10] Visualize  your solution given the input data provided above and again using the `RK45` integrator. Make four subplots

1. Plot the concentrations of the Nuclides against time showing the actual steps taken by the integrator as well as smooth solution from the dense output (hint: see the documentation for [solve_ivp](https://docs.scipy.org/doc/scipy/reference/generated/scipy.integrate.solve_ivp.html) and [this](https://stackoverflow.com/questions/36699155/how-to-get-color-of-most-recent-plotted-line-in-pythons-plt) to match colors.  Do not forget to provide the appropriate labels.
- Also plot $\Delta t(t)$ used by the adaptive time stepper and list the maximum number of steps taken
- Which eigenvalue controls stability?  For this eigenvalue, calculate $z=\lambda \Delta t$ at each time-step and plot on the stability diagram for Taylor-4 and Taylor-5 (which are equivalent to the stability plots for RK4 and RK5). Which order scheme is controlling the stability?



In [ ]:
# YOUR CODE HERE
raise NotImplementedError()

YOUR ANSWER HERE

**(c)** [4] Now let's make this problem stiff by introducing a very fast decaying second nuclide.  repeat the exercise and figures but now using $K_2=1000.$ 

In [ ]:
# YOUR CODE HERE
raise NotImplementedError()

YOUR ANSWER HERE

**(d)**  [4] Repeat the stiff problem but choose any method appropriate for stiff equations in `solve_ivp`.  Provide a brief description of how the solver is choosing time steps.  Feel free to try different methods

In [ ]:
# YOUR CODE HERE
raise NotImplementedError()

YOUR ANSWER HERE

**(e)** [4] Write a fixed time step BDF-2 scheme to solve this problem.  (hint:  this is a LMM, you'll need to figure out how to get started). you can use numpy.linalg.solve to solve the implicit linear system.  

In [ ]:
def BDF2(A, t_span, u_0, N):
    """
    uses a BDF2 scheme to solve the autonomous linear dynamical system u' = Au, u(0) = u_0
    with f(U) = Au
    
         3 U_{n+2} - 4 U_{n+1} + U_n = 2 \Delta t f(U_{n+2}) \\
         
    Parameters:
    -----------
    A: ndarray  
        m x m matrix of Coefficients such that F(u) = Au where m =len(u0)
    t_span: tuple, arraylike
        time span over which to integrate [t_min, t_max]
    u_0:  ndarray (len(n))
        array of initial conditions
    N: integer
        number of time steps
        
    Returns:
    --------
    t: numpy array, 
        actual time-steps take
    u:  ndarray of shape m x N
        solution array
    
    """
    # YOUR CODE HERE
    raise NotImplementedError()

In [ ]:
K = [1., 2., 3.]
A = setA(K)
N = 100
t_span = [0., 10.]
u_0 = numpy.array([1., 0., 0., 0.])
t, u = BDF2(A, t_span, u_0, N)
sol = solve_decay_system(t_span, u_0, K, method='BDF')

numpy.testing.assert_allclose(u, sol.sol(t),rtol=10., atol=1.e-3)
print('success!')

**(f)** [10] Graphically compare your solution to solve_ivp. 
* Make a plot that overlays your solution with that of solve_ivp
* Make a convergence plot that calculates the maximum absolute error between your solution and solve_ivp as a function of $\Delta t$ for $N = 2^n$ for $n\in[5,13]$.  Comment on the result.
* Do this for both 
$$
    K_1 = 1.,~~ K_2 = 2.,~~ K_3 = 3.
$$ 

and the stiff problem with $K_2 = 1000.$

In [ ]:
# YOUR CODE HERE
raise NotImplementedError()

YOUR ANSWER HERE

**Extra-Extra-Credit** write an adaptive time-stepper using a combination of BDF-2 and BDF-3 and compare your solution to solve_ivp (you'll also need to figure out how to use interpolation to adjust the time step).  Full disclosure:  I haven't done this yet but it should be doable